This project is made for Reasoning Agents part where the fundamental idea is to apply Reinforcement Learning to Mario game.

## Libraries
We have included several libraries which will be needed for the project. They are Gym, pandas, numpy, tensorflow and others.

In [1]:
import tkinter as tk
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import gym
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
import gym_super_mario_bros
from gym_super_mario_bros.actions import RIGHT_ONLY
from nes_py.wrappers import JoypadSpace
from IPython.display import clear_output

from keras.models import save_model, load_model
import time


## Environment

In [2]:
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, RIGHT_ONLY)

## Simple test
This is used to test the game and see how it exactly works.

In [3]:
# total_reward = 0
# done = True
# for step in range(100000):
#     env.render()
#     if done:
#         state = env.reset()
#     state, reward, done, info = env.step(env.action_space.sample())
#     preprocess_state(state)
#     print(info)
#     total_reward += reward
#     clear_output(wait=True)
# env.close()

## Class for the Mario

In [4]:
state = env.reset()
#preprocess_state(state)

In [5]:
class MarioAgent:
    def __init__(self, state_size, action_size):
        # we need to create variables for our agent here
        self.state_space = state_size
        self.action_space = action_size
        self.memory = deque(maxlen=5000)
        self.gamma = 0.8
        self.chosenAction = 0
        #giving the epsilon value for exploration and exploiation
        self.epsilon = 0.6
        self.max_epsilon = 1
        self.min_epsilon = 0.01
        self.decay_epsilon = 0.0001

        # we then need to build the NN for the agent
        self.main_network = self.build_network()
        self.target_network = self.build_network()
        self.update_target_network()

    def build_network(self):
        model = Sequential()
        model.add(Conv2D(64, (4,4), strides=4, padding='same',input_shape=self.state_space))
        model.add(Activation('relu'))

        model.add(Conv2D(64, (4,4), strides=2, padding='same'))
        model.add(Activation('relu'))

        model.add(Conv2D(64, (3,3), strides=1, padding='same'))
        model.add(Activation('relu'))
        model.add(Flatten())

        model.add(Dense(512, activation='relu'))
        model.add(Dense(256, activation='relu'))
        model.add(Dense(self.action_space, activation='linear'))

        model.compile(loss='mse', optimizer=Adam())
        return model

    def update_target_network(self):
        self.target_network.set_weights(self.main_network.get_weights())

    def act(self, state, onGround):
        if onGround < 83:
            if random.uniform(0,1) < self.epsilon:
                self.chosenAction = np.random.randint(self.action_space)
                return self.chosenAction
            Q_value = self.main_network.predict(state)
            self.chosenAction = np.argmax(Q_value[0])
            return self.chosenAction
        else:
            return self.chosenAction


    def update_epsilon(self, episode):
        self.epsilon = self.min_epsilon + (self.max_epsilon - self.min_epsilon)*np.exp(-self.decay_epsilon * episode)

    def train(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)

        for state, action, reward, next_state, done in minibatch:
            target = self.main_network.predict(state)
            print(target)
            if done:
                target[0][action] = reward
            else:
                target[0][action] = (reward+ self.gamma * np.amax(self.target_network.predict(next_state)))

            self.main_network.fit(state, target, epochs=1, verbose=0)
    def store_transition(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def get_pred_act(self, state):
        Q_values = self.main_network.predict(state)
        return np.argmax(Q_values[0])
    def load(self, name):
        self.main_network = load_model(name)
        self.target_network = load_model(name)

    def save(self, name):
        save_model(self.main_network, name)

In [6]:
action_space = env.action_space.n
state_space = (80,88,1)
env.observation_space
from PIL import Image

def preprocess_state(state):
    image = Image.fromarray(state)
    image = image.resize((88,80))
    image = image.convert('L')
    image = np.array(image)
    return image

In [7]:
num_episodes = 1000
num_timesteps = 400000
batch_size = 64
DEBUG_LEN = 300

In [8]:
mario = MarioAgent(state_space, action_space)

In [9]:
stuck_buffer = deque(maxlen=DEBUG_LEN)
for i in range(num_episodes):
    Return = 0
    done = False
    time_step = 0
    onGround = 79

    state = preprocess_state(env.reset())
    state = state.reshape(-1, 80, 88, 1)

    for t in range(num_timesteps):
        #env.render()
        time_step += 1

        if t>1 and stuck_buffer.count(stuck_buffer[-1]) == DEBUG_LEN-50:
            action = mario.act(state, onGround=79)
        else:
            action = mario.act(state, onGround)

        action = mario.act(state, onGround)
        next_state, reward, done, info = env.step(action)
        onGround = info['y_pos']
        stuck_buffer.append(info['x_pos'])
        
        # we can also get the values of other type of info as said here: https://pypi.org/project/gym-super-mario-bros/
        # for instance, info['coins'], info['flag_get'], info['score'], info['time'], 

        next_state = preprocess_state(next_state)
        next_state = next_state.reshape(-1, 80,88,1)

        mario.store_transition(state, action, reward, next_state, done)
        state = next_state
        Return += reward
        print("Episode is: {}\nTotal Time Step: {}\nCurrent Reward: {}\nEpsilon is: {}".format(str(i), str(time_step), str(Return), str(mario.epsilon)))

        clear_output(wait=True)

        if done:
            break
        if len(mario.memory) > batch_size:
            mario.train(batch_size)
    mario.update_epsilon(i)
    clear_output(wait=True)
    mario.update_target_network()
env.close()

[[6.7822337 7.8631315 9.27236   7.592083  7.8924994]]
[[6.7846274 7.79663   9.307657  7.6307054 7.825416 ]]
[[6.5994334 7.746745  9.266344  7.4901605 7.8272495]]
[[6.603382  7.7605014 9.416162  7.5309153 7.8984256]]
[[6.8817296 8.000079  9.8769    7.8090687 8.205997 ]]
[[ 6.998809   8.004982  10.058356   7.8904066  8.36546  ]]
[[ 7.1379366  8.101027  10.150245   7.8725424  8.344995 ]]
[[ 7.2056346  8.099422  10.201339   7.8589015  8.191178 ]]
[[7.07005   7.9234304 9.978672  7.6634645 7.747539 ]]
[[6.849739  7.683754  9.661008  7.401821  7.1625347]]
[[6.854863 7.612127 9.62449  7.336837 6.797142]]
[[6.505857  7.298324  9.189573  7.075127  6.2531123]]
[[6.3995776 7.1890793 9.058817  7.0110993 5.930677 ]]
[[6.4397764 7.2014923 9.100327  7.0724807 5.8670683]]
[[6.784896  7.456399  9.504455  7.3666115 6.1200337]]
[[6.962476  7.562263  9.629881  7.4944053 6.280153 ]]
[[7.3476934 7.7406664 9.977095  7.732374  6.4583883]]
[[ 7.7118163  8.032207  10.363714   8.008423   6.717735 ]]
[[ 8.14333   

KeyboardInterrupt: 

## Save model 

In [10]:
mario.save('mario-v1.h5')

## Load model

In [11]:
mario.load('mario-v1.h5')

In [14]:
while 1:
    done = False
    state = preprocess_state(env.reset())
    state = state.reshape(-1, 80,88,1)
    total_reward = 0
    onGround = 79
    
    while not done:
        env.render()
        action = mario.act(state, onGround)
        next_state, reward, done, info = env.step(action)
        
        onGround = info['y_pos']
        print(info)
        next_state = preprocess_state(next_state)
        next_state = next_state.reshape(-1, 80, 88, 1)
        state = next_state
env.close()


{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 400, 'world': 1, 'x_pos': 40, 'y_pos': 79}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 400, 'world': 1, 'x_pos': 40, 'y_pos': 84}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 400, 'world': 1, 'x_pos': 40, 'y_pos': 89}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 400, 'world': 1, 'x_pos': 40, 'y_pos': 93}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 400, 'world': 1, 'x_pos': 40, 'y_pos': 98}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 400, 'world': 1, 'x_pos': 41, 'y_pos': 102}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 400, 'world': 1, 'x_pos': 41, 'y_pos': 105}
{'coins': 0, 'flag_get': False, 'life':

{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 397, 'world': 1, 'x_pos': 162, 'y_pos': 162}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 397, 'world': 1, 'x_pos': 165, 'y_pos': 163}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 397, 'world': 1, 'x_pos': 168, 'y_pos': 164}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 397, 'world': 1, 'x_pos': 171, 'y_pos': 164}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 397, 'world': 1, 'x_pos': 174, 'y_pos': 165}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 397, 'world': 1, 'x_pos': 177, 'y_pos': 165}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 397, 'world': 1, 'x_pos': 180, 'y_pos': 165}
{'coins': 0, 'flag_get': Fa

{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 394, 'world': 1, 'x_pos': 347, 'y_pos': 82}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 394, 'world': 1, 'x_pos': 350, 'y_pos': 79}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 394, 'world': 1, 'x_pos': 353, 'y_pos': 85}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 394, 'world': 1, 'x_pos': 356, 'y_pos': 91}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 394, 'world': 1, 'x_pos': 359, 'y_pos': 96}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 394, 'world': 1, 'x_pos': 362, 'y_pos': 100}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 394, 'world': 1, 'x_pos': 365, 'y_pos': 104}
{'coins': 0, 'flag_get': False, 

{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 391, 'world': 1, 'x_pos': 483, 'y_pos': 96}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 391, 'world': 1, 'x_pos': 485, 'y_pos': 91}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 391, 'world': 1, 'x_pos': 487, 'y_pos': 86}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 390, 'world': 1, 'x_pos': 488, 'y_pos': 81}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 390, 'world': 1, 'x_pos': 490, 'y_pos': 79}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 390, 'world': 1, 'x_pos': 492, 'y_pos': 79}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 390, 'world': 1, 'x_pos': 494, 'y_pos': 84}
{'coins': 0, 'flag_get': False, 'l

{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 387, 'world': 1, 'x_pos': 594, 'y_pos': 92}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 387, 'world': 1, 'x_pos': 594, 'y_pos': 87}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 387, 'world': 1, 'x_pos': 594, 'y_pos': 82}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 387, 'world': 1, 'x_pos': 594, 'y_pos': 79}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 387, 'world': 1, 'x_pos': 594, 'y_pos': 84}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 387, 'world': 1, 'x_pos': 594, 'y_pos': 89}
{'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 387, 'world': 1, 'x_pos': 594, 'y_pos': 93}
{'coins': 0, 'flag_get': False, 'l

...

{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 276, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 276, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 276, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 275, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 275, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 275, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 275, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coin

{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 272, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 272, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 272, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 272, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 272, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 272, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 272, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coin

{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 269, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 269, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 269, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 269, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 269, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 269, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 269, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coin

{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 266, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 266, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 266, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 266, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 266, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 266, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coins': 1, 'flag_get': False, 'life': 1, 'score': 300, 'stage': 1, 'status': 'small', 'time': 266, 'world': 1, 'x_pos': 2178, 'y_pos': 127}
{'coin

KeyboardInterrupt: 